Predicción y Decisión de umbral
#Analytics y Big Data: Ciencia de los Datos aplicada al mundo de los negocios

Aplicaremos el mejor modelo encontrado con los datos de crédito (SVM) sobre nuevos clientes que no sabemos su comportamiento. De esta forma emulamos lo que sería una eventual aplicación de un modelo predictivo.

Primero importamos las librerías a utilizar:

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas.util.testing as tm
from joblib import dump, load

Volvemos a montar google drive para que reconozca nuestra base de datos:

**Cuando corra la siguiente linea, le pedira un código, por favor siga las intrucciones a continuación, inserte el código entregado y presione enter**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Cargamos el archivo con la base de datos de testeo (segunda hoja del excel):

In [ ]:
ruta_data_set = 'gdrive/My Drive/Colab Notebooks/Base_Fuga_DABI_M4.xlsx'
data_set = pd.read_excel(ruta_data_set, sheet_name = 'Validación')

Cargamos ahora el archivo con el modelo entrenado:

In [ ]:
modelo_entrenado = load('gdrive/My Drive/Colab Notebooks/modelo_entrenado_RF.joblib')

Nuestro objeto data_set tiene las variables independientes que nos ayudarán a predecir la variable objetivo (si cliente cae en default o no).

Antes de aplicar el modelo, debemos replicar el preprocesamiento a este nuevo *set* de datos.

#Preprocesamiento de datos de predicción

Veamos que contiene nuestra base de datos de predicción:


In [ ]:
print(data_set)

      CreditMes_T  CreditMes_T-1  CreditMes_T-2  NumTarjCred_T  \
0       1029650.0         597200         562100              1   
1       1102100.0         626000         606650              2   
2        995000.0         603950         565250              1   
3        990500.0         597200         562550              0   
4       1000400.0         604400         567050              0   
...           ...            ...            ...            ...   
4439    1003550.0         615200         570200              2   
4440     996350.0         597650         562100              1   
4441    1033700.0         698900         702500              1   
4442    1035500.0         621050         565250              2   
4443     990950.0         596750         561650              0   

      NumTarjCred_T-1  NumTarjCred_T-2  Ingreso  Edad  NumTransWeb_T  \
0                   1                1  1941150  57.0              0   
1                   2                2   887600  31.0          

In [ ]:
data_set.describe(include='all')

,CreditMes_T,CreditMes_T-1,CreditMes_T-2,NumTarjCred_T,NumTarjCred_T-1,NumTarjCred_T-2,Ingreso,Edad,NumTransWeb_T,NumTransWeb_T-1,...,MargenComp_T-2,MargenComp_T-3,MargenComp_T-4,MargenComp_T-5,MargenComp_T-6,Telefono,NivelEduc,Genero,EstCivil,Region
count,4.442000e+03,4.444000e+03,4.444000e+03,4444.000000,4444.000000,4444.000000,4.444000e+03,4441.000000,4444.000000,4444.000000,...,4444.000000,4444.000000,4444.000000,4444.000000,4444.000000,4444,4444,4442,4444,4444
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,4,2,4,1
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SI,UNI,M,CAS,RM
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4444,2861,3101,2377,4444
mean,1.020620e+06,6.315683e+05,5.838441e+05,0.890189,0.889064,0.896265,1.246531e+06,36.011484,5.403015,4.025653,...,226234.243924,253062.799280,253086.156616,260942.074707,196048.987399,NaN,NaN,NaN,NaN,NaN
std,7.979266e+04,8.886365e+04,6.354921e+04,0.740825,0.748065,0.759837,6.153649e+05,8.976727,11.773686,8.645611,...,2982.142499,1311.480598,3436.618012,2661.500733,3985.175021,NaN,NaN,NaN,NaN,NaN
min,5.000000e+05,5.382500e+05,5.360000e+05,0.000000,0.000000,0.000000,5.000000e+05,19.000000,0.000000,0.000000,...,100000.000000,221340.000000,123580.000000,100000.000000,100000.000000,NaN,NaN,NaN,NaN,NaN
25%,9.914000e+05,5.976500e+05,5.625500e+05,1.000000,1.000000,1.000000,8.933000e+05,29.000000,0.000000,0.000000,...,226000.000000,252840.000000,252960.000000,260800.000000,195380.000000,NaN,NaN,NaN,NaN,NaN
50%,9.995000e+05,6.066500e+05,5.670500e+05,1.000000,1.000000,1.000000,1.069050e+06,34.000000,1.000000,1.000000,...,226120.000000,252980.000000,253080.000000,260900.000000,195660.000000,NaN,NaN,NaN,NaN,NaN
75%,1.016600e+06,6.255500e+05,5.787500e+05,1.000000,1.000000,1.000000,1.386588e+06,41.000000,6.000000,5.000000,...,226440.000000,253260.000000,253360.000000,261120.000000,196460.000000,NaN,NaN,NaN,NaN,NaN


La estadística descriptiva indica que tenemos un ingreso cero (inconsistencia) y algunos valores perdidos:

In [ ]:
data_set.loc[data_set['Ingreso'] < 1, ['Ingreso']] = np.nan
data_set.loc[data_set['Edad'] > 100, ['Edad']] = np.nan

Remplazamos los valores perdidos por la media:

In [ ]:
# VP/I menores al 5%, siendo también MCAR: reemplazo por la mediana al ser variables numéricas, y por presentar grados altos de dispersión
# (la mediana es más robusta que la media respecto a la dispersión de los datos)
data_set['CreditMes_T'].fillna(data_set['CreditMes_T'].median(skipna =True), inplace=True)
data_set['NumTarjCred_T'].fillna(data_set['NumTarjCred_T'].median(skipna =True), inplace=True)
data_set['NumTarjCred_T-1'].fillna(data_set['NumTarjCred_T-1'].median(skipna =True), inplace=True)
data_set['NumTransWeb_T'].fillna(data_set['NumTransWeb_T'].median(skipna =True), inplace=True)
data_set['NumTransWeb_T-1'].fillna(data_set['NumTransWeb_T-1'].median(skipna =True), inplace=True)
data_set['MargenComp_T'].fillna(data_set['MargenComp_T'].median(skipna =True), inplace=True)
data_set['MargenComp_T-1'].fillna(data_set['MargenComp_T-1'].median(skipna =True), inplace=True)
data_set['MargenComp_T-4'].fillna(data_set['MargenComp_T-4'].median(skipna =True), inplace=True)
data_set['MargenComp_T-5'].fillna(data_set['MargenComp_T-5'].median(skipna =True), inplace=True)
data_set['Ingreso'].fillna(data_set['Ingreso'].median(skipna =True), inplace=True)

# VP/I menores al 5%, siendo también MCAR: reemplazo por la moda al ser variables categóricas.
mode1 = data_set['NivelEduc'].mode()
data_set['NivelEduc'].fillna(mode1[0], inplace=True)

mode2 = data_set['EstCivil'].mode()
data_set['EstCivil'].fillna(mode2[0], inplace=True)

Creamos la variable de endeudamiento (ratio deuda total sobre ingreso)


In [ ]:
# VP/I menores al 5%, siendo también NMAR: reemplazo ad-hoc.
data_set['Edad'].fillna(100, inplace=True)

Ahora que no existen inconsistencias en nuestra base de datos, transformamos las variables financieras a su logaritmos (debemos hacer EXACTAMENTE lo mismo que en el modelo original).

In [ ]:
data_set['Genero'] = data_set['Genero'].replace('H', 'M')
data_set['NivelEduc'] = data_set['NivelEduc'].replace('EST_UNI', 'UNI')

In [ ]:
data_set['Credito_Total'] = data_set['CreditMes_T'] + data_set['CreditMes_T-1'] + data_set['CreditMes_T-2'] # Suma de todos los creditos
data_set['Credito_Ing'] = data_set['Credito_Total']/data_set['Ingreso'] # Ratio credito respecto al ingreso
data_set['NumTarjCred_Total'] = data_set.loc[:, 'NumTarjCred_T':'NumTarjCred_T-2'].max(axis=1) # Max de tarjetas de credito por cliente
data_set['NumTransWeb_Total'] = data_set['NumTransWeb_T'] + data_set['NumTransWeb_T-1'] + data_set['NumTransWeb_T-2'] # Suma de transacciones
data_set['MargenComp_Total'] = data_set['MargenComp_T'] + data_set['MargenComp_T-1'] + data_set['MargenComp_T-2'] # Suma de margenes, se descartan meses de T-3 a T-6

del data_set['CreditMes_T']
del data_set['CreditMes_T-1']
del data_set['CreditMes_T-2']

del data_set['NumTarjCred_T']
del data_set['NumTarjCred_T-1']
del data_set['NumTarjCred_T-2']

del data_set['NumTransWeb_T']
del data_set['NumTransWeb_T-1']
del data_set['NumTransWeb_T-2']

del data_set['MargenComp_T']
del data_set['MargenComp_T-1']
del data_set['MargenComp_T-2']
del data_set['MargenComp_T-3']
del data_set['MargenComp_T-4']
del data_set['MargenComp_T-5']
del data_set['MargenComp_T-6']


In [ ]:
data_set['Ln_Ingreso'] = np.log(data_set['Ingreso']+1)
data_set['Ln_Credito_Total'] = np.log(data_set['Credito_Total']+1)

del data_set['Ingreso']
del data_set['Credito_Total']

In [ ]:
data_set

,Edad,Telefono,Region,Credito_Ing,NumTarjCred_Total,NumTransWeb_Total,MargenComp_Total,Ln_Ingreso,Ln_Credito_Total,Genero_M,NivelEduc_TEC,NivelEduc_UNI,EstCivil_DIV,EstCivil_SOL,EstCivil_VIU
0,57.0,SI,RM,1.127656,1,1,572700.0,14.478792,14.598933,1,0,1,0,0,0
1,31.0,SI,RM,2.630408,2,30,631360.0,13.696278,14.663416,1,0,1,0,1,0
2,55.0,SI,RM,0.866096,1,7,632100.0,14.731322,14.587562,1,1,0,0,0,0
3,29.0,SI,RM,1.561717,0,2,709400.0,14.135310,14.581095,1,0,1,0,1,0
4,33.0,SI,RM,1.852166,0,29,627240.0,13.974735,14.591090,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439,33.0,SI,RM,1.285576,2,8,656920.0,14.347726,14.598933,1,0,1,0,0,0
4440,35.0,SI,RM,1.516778,1,0,630140.0,14.167224,14.583812,1,1,0,0,0,0
4441,41.0,SI,RM,1.516440,1,0,627540.0,14.289133,14.705499,1,0,1,0,0,0
4442,29.0,SI,RM,1.441604,2,20,629420.0,14.248073,14.613829,1,0,1,0,0,0


In [ ]:
del data_set['Edad']
del data_set['Telefono']
del data_set['Region']
del data_set['Ln_Ingreso']
del data_set['Genero_M']
del data_set['NivelEduc_UNI']
del data_set['EstCivil_DIV']
del data_set['EstCivil_VIU']

In [ ]:
data_set

,Credito_Ing,NumTarjCred_Total,NumTransWeb_Total,MargenComp_Total,Ln_Credito_Total,NivelEduc_TEC,EstCivil_SOL
0,1.127656,1,1,572700.0,14.598933,0,0
1,2.630408,2,30,631360.0,14.663416,0,1
2,0.866096,1,7,632100.0,14.587562,1,0
3,1.561717,0,2,709400.0,14.581095,0,1
4,1.852166,0,29,627240.0,14.591090,0,0
...,...,...,...,...,...,...,...
4439,1.285576,2,8,656920.0,14.598933,0,0
4440,1.516778,1,0,630140.0,14.583812,1,0
4441,1.516440,1,0,627540.0,14.705499,0,0
4442,1.441604,2,20,629420.0,14.613829,0,0


Creamos ahora un nuevo dataframe, X_pre, que tendrá las 7 variables relavantes que incluía el modelo de SVM.

In [ ]:
X_pre = pd.DataFrame(data_set[['Credito_Ing','NumTarjCred_Total','NumTransWeb_Total','MargenComp_Total','Ln_Credito_Total','NivelEduc_TEC','EstCivil_SOL']])

In [ ]:
X_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4444 entries, 0 to 4443
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Credito_Ing        4444 non-null   float64
 1   NumTarjCred_Total  4444 non-null   int64  
 2   NumTransWeb_Total  4444 non-null   int64  
 3   MargenComp_Total   4444 non-null   float64
 4   Ln_Credito_Total   4444 non-null   float64
 5   NivelEduc_TEC      4444 non-null   uint8  
 6   EstCivil_SOL       4444 non-null   uint8  
dtypes: float64(3), int64(2), uint8(2)
memory usage: 182.4 KB


Normalizamos la base de datos:

In [ ]:
scaler = MinMaxScaler()
scaler_data = scaler.fit(X_pre)
X = pd.DataFrame(scaler_data.transform(X_pre), index= X_pre.index, columns= X_pre.columns)

#Predicción

Con el modelo de entrenamiento y las variables independientes ya definidas y procesadas, podemos realizar la predicción de la variable objetivo como se muestra a continuación:

In [ ]:
prediccion = modelo_entrenado.predict(X)

Podemos ver los resultados de predicción si imprimimos el objeto "prediccion", que toma el valor 1 cuando el cliente se predice como mal pagador ("default" o "s") y 0 en caso contrario ("no default" o "n").

In [ ]:
print(prediccion)

[1 1 1 ... 1 1 1]


Para realizar esta predicción, el modelo utiliza un umbral de 0.5 para la probabilidad de incumplimiento. Esto quiere decir que si p>0.5 "prediccion" toma el valor de 1, y cuando sea menor a este umbral toma el valor de 0.

Pero cual es la mejor decisión de umbral? Contamos la cantidad de clientes predichos que caen en default:

In [ ]:
prediccion.sum()/prediccion.size

0.9997749774977498

Recordemos que la tasa de clientes malos es de un 20% en la base original. Nuestro modelo predice, sin embargo, más de un 38% de clientes malos pagadores. Esto se debe a la corrección que hace SMOTE, que le hace creer al modelo que hay la misma cantidad tanto de clientes buenos como malos.

Podemos modificar este umbral según lo que necesitamos, definiendo por ejemplo el umbral de 0.6 para tener una política de otorgamiento menos estricta. Una política muy restrictiva puede hacernos perder participación de mercado.

In [ ]:
prediccion_2 = (modelo_entrenado.predict_proba(X)[:,1] >= 0.6).astype(int)

In [ ]:
print(prediccion_2)

[1 1 1 ... 1 1 1]


Nuevamente contamos la cantidad de defaulters con este nuevo umbral:

In [ ]:
prediccion_2.sum()/prediccion_2.size

0.9914491449144914

Podemos ver que los *defaulters* disminuyen al aumentar el umbral (desde un 39% a un 31% aproximadamente). Es posible modificar este criterio según los costos y beneficios del modelo.

#Exportar datos a excel

Finalmente, armamos la base de datos con todas las variables originales y le agregamos la variable predicha para tener toda la información del cliente y luego exportar a excel estos datos y poder trabajar con ellos.

Primero tenemos que transformar las variables que transformamos a logaritmos a su escala original

In [ ]:
data_set['Ingreso'] = np.exp(data_set['Ln_Ingreso'])-1
data_set['Credito_Total'] = np.exp(data_set['Ln_Credito_Total'])-1

del data_set['Ln_Ingreso']
del data_set['Ln_Credito_Total']

Ahora con las variables listas agregamos la variable de predicción a nuestra base de datos

In [ ]:
data_set['VarObj'] = prediccion_2

Por último, exportamos la base de datos a excel y la guardamos en nuestro drive:

In [ ]:
data_set.to_excel('Base_Fuga_DABI_M4_Prediccion.xlsx')
!cp Base_Fuga_DABI_M4_Prediccion.xlsx "gdrive/My Drive/Colab Notebooks/"